## Computer Vision Simplified Trainer

This notebook is modified from the `00-computer-vision-trainer.ipynb` notebook and has the explanation text removed

The basic steps we'll take are:

1. Importing our collected, organized, and cleaned images
1. Fine-tune a pretrained neural network model to take images as inputs and output category labels
1. Examining our output and deciding if we're happy with the model

## STOP HERE, THIS IS THE BASE TRAINER, MAKE A COPY AND DO YOUR WORK THERE

Describe what you want to do for this training here:

Item Transforms:

Pre-Trained Model: 

Batch Size:  

Validation Size: 

Epochs: 


## STEP 0: Ensure you are running this notebook within a Docker Container

In [ ]:
path = '/workspace/data/lakota_data'

## STEP 1: Ensure you have spectrogram images

The cell below will let you know what directories and file numbers are available in `/workspace/data/lakota_data`. This value should be assigned in an earlier cell for the variable `path`.

In [ ]:
import os

if not os.path.isdir(path):
    print(f"'{path}' is not a valid directory.")
else:
    print(f"Scanning directory: {path}\n")
    for entry in os.scandir(path):
        if entry.is_dir():
            subdir_path = entry.path
            file_count = sum(
                1 for item in os.scandir(subdir_path) if item.is_file()
            )
            print(f"{entry.name}/ — {file_count} files")

## Step 2: Train an initial model so that we clean our data

Choose a model. More information about pretrained models, and others that you may want to use instead of `resnet34.a1_in1k` can be found at  
[https://www.kaggle.com/code/jhoward/which-image-models-are-best](https://www.kaggle.com/code/jhoward/which-image-models-are-best).

In [ ]:
from fastai.vision.all import *
import timm

The three cells below will support you in choosing a pre-trained model to use. If you're ok with using the default model of `resnet34.a1_in1k`, or already have one that you want to use, you can skip to the `DataBlock` cell.

In [ ]:
timm.list_models(pretrained=True)

In [ ]:
timm.list_models('resnet*', pretrained=True)

In [ ]:
# Model name
model_name = 'resnet34.a1_in1k'

# Create the model instance
model = timm.create_model(model_name, pretrained=True)

# Retrieve the default configuration
default_cfg = model.default_cfg
print(default_cfg)

Whichever model you choose, ensure that the `item_tfms` resize value matches the `'input_size'` in the information above. For the `'resnet34.a1_in1k'` model, the input size is `(3, 224, 224)` and so we need to ensure that the value below for `item_tfms` is:  
  
`item_tfms=Resize(224, 'squish')`

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(224, 'squish'),
    batch_tfms=[]#We don't want to transform our spectrograms, leave this line alone
).dataloaders(path, bs=50)

**NOTE: The batch size above (set by `bs=50`) cannot be higher than the number of image files you have.**

If the batch size is too high, you will see an error for the cell below that says  
`ValueError: This DataLoader does not contain any batches`.  

Reduce the `bs` value above in order to overcome this issue.

If you feel the batch size is good, save it in the cell below, we will use this in the longer training.

In [ ]:
bs = 50 #this should match the value you have in the DataLoader above.

In [ ]:
dls.show_batch(max_n=8, nrows=2, ncols=4)

In [ ]:
learn = vision_learner(dls, model_name, metrics=error_rate)

In [ ]:
learn.lr_find() # find the learning rate

In [ ]:
learning_rate = 0.00100000004749745134 # use the value from the lr_find() method above

In [ ]:
initial_training_epochs = 2
learn.fine_tune(initial_training_epochs, base_lr=learning_rate) 

The point of this initial training is to ensure that the longer model training below will work correctly.

**NOTE: If you get an error that states `RuntimeError: CUDA error: out of memory`, reduce the batch size value `bs=` in the `dataloaders()` parameters above.**

In [ ]:
learn.show_results()

## Step 3: Let's see how our model did and what it had the most difficulty with

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(normalize=True, title='Confusion Matrix', figsize=(12, 12))

In [ ]:
interp.plot_top_losses(9, figsize=(15,10))

## Step 4: Now We Train A Model Using What We've Learned

Set the parameter of your training

Naming convention: purpose_modelname_errorrate

What the model does?
What the model used?
How good is the model?

In [ ]:

#first set how often the model should save a checkpoint file, these will be stored as .pth files, not .pkl
# not that .pth files will contain the model architecture, whereas .pkl files do not, which means .pth files are much larger in size

last_epoch_save = 0 # leave this value as zero if you are starting over from cleaned data

save_after_this_many_epochs = 5 # a .pth checkpoint file will be created every 5 epochs

model = model_name # this was set above

path = path # ensure this points to your data

# Define the number of additional epochs you want to train for

additional_epochs = 10  # Set the total number of epochs to run in the cell below

learning_rate = learning_rate # use the value from the lr_find() method above

dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(224, 'squish'),#RandomResizedCrop(224, min_scale=0.3),
    batch_tfms=[]
).dataloaders(path, bs=bs)

In [ ]:
from fastai.callback.core import Callback
from fastai.learner import load_learner
from fastai.data.core import DataLoaders

# SaveModelCallback will be triggered on each epoch, The default is 10 epochs, but you can set the amount using the save_after_this_many_epochs variable
class SaveModelCallback(Callback):
    def __init__(self, every=10, path='model', last_epoch_save=0):
        self.every = every
        self.path = path
        self.last_epoch_save = last_epoch_save

    def after_epoch(self):
        "Save the model every `self.every` epochs"
        if (self.epoch + 1) % self.every == 0:
            self.learn.save(f'{self.path}_epoch{self.epoch + (1+self.last_epoch_save)}')

# Define your data recover function, this function will return .pkl model from one of your checkpoint save files in the .pth format
# path = the path to your data (e.g. /workspace/data/TRAINING_DATA), path_to_pth_model = the path to your last saved .pth file
def recover_dl_from_pth(path, path_to_pth_model):
    dls = ImageDataLoaders.from_folder(path)
    learn = cnn_learner(dls, model) # be sure to use the same pretrained model here that you did for previous training, the model variable set above
    learn.load(path_to_pth_model)  # Load the model from the .pth file into the learn variable
    return learn # return the learn variable

# Load the previously trained model
recover_from_pth_file = False

if(recover_from_pth_file):
    # The code below will create a learner recovered from a checkpoint save
    # You will want to make sure that the pretrained model in recover_dl_from_pth() the same as what was used for fine tuning earlier
    learn_long = recover_dl_from_pth(path, '/workspace/models/model_epochEPOCH_NUM_HERE_TO_GET_TO_CORRECT_FLE.pth')
else:
    # This code will start at 0 or recover from a .pkl file
    if(last_epoch_save > 0):
        previous_model_path = '/workspace/models/MODEL_FILENAME.pkl'  # replace with your actual model path
        learn_long = load_learner(previous_model_path) #note that this is creating a different learner than the one we used above
    else:
        learn_long = vision_learner(dls, model, metrics=error_rate) # create the learner
    # Ensure data loaders are set up
    learn_long.dls = dls # path is the path to our data, which was set earlier

# Instantiate the custom callback
# modify last_epoch_save to be the number of epochs the model was trained up to
save_model_callback = SaveModelCallback(every=save_after_this_many_epochs, path='/workspace/models/model', last_epoch_save = last_epoch_save)

# Continue training the model with the custom callback
learn_long.fine_tune(additional_epochs, base_lr=learning_rate, cbs=[save_model_callback])

# Save the updated model at the end
updated_model_path = f'/workspace/models/model_epoch{(last_epoch_save+additional_epochs)}.pkl'
learn_long.export(updated_model_path)

print("Training completed and model saved.")


In [ ]:
interp_long = ClassificationInterpretation.from_learner(learn_long)
interp_long.plot_confusion_matrix(normalize=True, title='Confusion Matrix', figsize=(12, 12))